In [1]:
%load_ext autoreload
%autoreload 2
import time
import tensorflow as tf
import p8_util

Using TensorFlow backend.


In [2]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
datadir = './data'
is_tensorboard = True

In [3]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (4134): Aucun processus de ce type
kill: (4143)

In [4]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump


#### Increase of train dataset against test dataset.

In [5]:
import numpy as np
x_train = np.concatenate([x_train, x_test[:27,:,:,:]], axis=0)
x_test = x_test[27:,:,:,:]
print(x_train.shape, x_test.shape)

y_train = np.concatenate([y_train, y_test[:27]])
y_test = y_test[27:]
print(y_train.shape, y_test.shape)

(441, 224, 224, 3) (20, 224, 224, 3)
(441, 3) (20, 3)


In [14]:
p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']

{'rnn_layer_num': 2, 'rnn_hidden_units': 128, 'rnn_timesteps': 224}

In [56]:
import p8_util_config
import NNAdaNetBuilder

#-----------------------------------------------------------------
# Get the number of convolutional layers for CNN network
# This will fixe the equivalent parameter of AdaNet num layers.
#-----------------------------------------------------------------
if p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'CNN' :
    layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['conv_layer_num']
elif p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'RNN' \
or   p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'GRU' \
or   p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'SGRU':
    layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['rnn_layer_num']

print("\n Number of convolutional layers= {}".format(layer_num))
oNNAdaNetBuilder = NNAdaNetBuilder.NNAdaNetBuilder(p8_util_config.dict_adanet_config, num_layers=layer_num)

oNNAdaNetBuilder.show()

print(p8_util_config.MAX_STEPS,p8_util_config.NUM_EPOCHS)


 Number of convolutional layers= 2

*** NNAdaNetBuilder : NN Type=SGRU


NN type              : ............................ SGRU
Units in dense layer : ............................ 10
Number of layers     : ............................ 2
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Weights initializer  : ............................ truncated_normal
Batch normalization  : ............................ True


Hidden units         : ............................ 128
Stacked cells        : ............................ 2
Time steps           : ............................ 224
200 6


In [57]:
# Parameters that will be provided to model_fn
import p8_util_config
nn_type = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']


params = {'net_builder':oNNAdaNetBuilder, 'nn_type':nn_type}
print("\n NN Type= {}".format(nn_type))
my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)  

classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=my_config)


 NN Type= SGRU

*** make_config() : output dir= ./tmp/baseline/SGRU
INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/SGRU', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9b9e610898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [58]:
oNNAdaNetBuilder._nb_class
oNNAdaNetBuilder._dict_rnn_layer_config['rnn_hidden_units']

128

In [59]:
train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train, p8_util_config.NUM_EPOCHS\
                          , tuple_dimension=tuple_dimension\
                          , batch_size=p8_util_config.BATCH_SIZE\
                          ),
        max_steps=p8_util_config.MAX_STEPS)

eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("test", x_test, y_test, p8_util_config.NUM_EPOCHS\
                           ,tuple_dimension=tuple_dimension\
                           ,batch_size=p8_util_config.BATCH_SIZE\
                           ),
        steps=None,
        throttle_secs=1)

In [60]:
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.

*** input_fn() : feature_shape= [224, 224, 3] / Label shape= [3]
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 3)

*** _build_rnn_subnetwork() : output[-1]= Tensor("rnn/rnn/multi_rnn_cell/cell_1/gru_cell/add_223:0", shape=(?, 128), dtype=float32) / Weight= <tf.Variable 'W:0' shape=(128, 3) dtype=float32_ref>

*** my_model_fn() : logits shape= (?, 3) / labels shape= (?, 3)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Sa

INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (20, 3) / label values= [0. 0. 1.]
INFO:tensorflow:Finished evaluation at 2019-05-08-18:41:54
INFO:tensorflow:Saving dict for global step 60: SGRU_Eval_accuracy = 0.3, global_step = 60, loss = 1.1713278
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 60: ./tmp/baseline/SGRU/model.ckpt-60
INFO:tensorflow:Saving checkpoints for 70 into ./tmp/baseline/SGRU/model.ckpt.

*** input_fn() : feature_shape= [224, 224, 3] / Label shape= [3]
feature_shape= [224, 224, 3]
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 3)

*** _build_rnn_subnetwork() : output[-1]= Tensor("rnn/rnn/multi_rnn_cell/cell_1/gru_cell/add_223:0", shape=(?, 128), dtype=float32) / Weight= <tf.Variable 'W:0' shape=(128, 3) dtype=float32_ref>

*** my_model_fn() : logits shape= (?, 3) / labels shape= (?, 3)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluatio


*** _build_rnn_subnetwork() : output[-1]= Tensor("rnn/rnn/multi_rnn_cell/cell_1/gru_cell/add_223:0", shape=(?, 128), dtype=float32) / Weight= <tf.Variable 'W:0' shape=(128, 3) dtype=float32_ref>

*** my_model_fn() : logits shape= (?, 3) / labels shape= (?, 3)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-08-18:43:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/SGRU/model.ckpt-130
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (20, 3) / label values= [0. 0. 1.]
INFO:tensorflow:Finished evaluation at 2019-05-08-18:44:01
INFO:tensorflow:Saving dict for global step 130: SGRU_Eval_accuracy = 0.5, global_step = 130, loss = 1.2056192
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 130: ./tmp/baseline/SGRU/model.ckpt-130

*** generator() : labels shape= (441, 3) / label values= [1. 0. 0.]
INFO:tensorflow:Saving checkpoin

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 190: ./tmp/baseline/SGRU/model.ckpt-190

*** generator() : labels shape= (441, 3) / label values= [1. 0. 0.]
INFO:tensorflow:Saving checkpoints for 200 into ./tmp/baseline/SGRU/model.ckpt.

*** input_fn() : feature_shape= [224, 224, 3] / Label shape= [3]
feature_shape= [224, 224, 3]
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 3)

*** _build_rnn_subnetwork() : output[-1]= Tensor("rnn/rnn/multi_rnn_cell/cell_1/gru_cell/add_223:0", shape=(?, 128), dtype=float32) / Weight= <tf.Variable 'W:0' shape=(128, 3) dtype=float32_ref>

*** my_model_fn() : logits shape= (?, 3) / labels shape= (?, 3)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-08-18:46:04
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/SGRU/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running l

In [61]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 543.5832524299622
SGRU_EVAL_ACCURACY: 0.3499999940395355
LOSS: 1.3505645990371704
GLOBAL_STEP: 200


In [62]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')